<img align="center" src="img/course.png" width="800">

# 16720 (B)  Object Tracking in Videos - Assignment 6 - Q1
    Instructor: Kris                          TAs: Wen-Hsuan (Lead), Zen, Yan, Rawal, Paritosh, Qichen

In [1]:
# Libraries

import numpy as np
from scipy.interpolate import RectBivariateSpline
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

## Q1: Lucas-Kanade Forward Additive Alignment for Tracking

### Overview
In this part, you will implement two variants of the Lucas-Kanade Tracking algorithm:

- The warp $W$ being translation only
- The warp $W$ being the full affine transformation

and evaluate them on the provided videos, which can be downloaded [here](https://www.dropbox.com/sh/l2ip26mkgf5p3e6/AACN2STT5Sk9r6bPeEXIYKZCa?dl=0). It is highly recommended that you finish the theory section first, or at the very least, go over the provided materials to gain a better understanding of the algorithms. You may also find these materials useful: [link](https://www.ri.cmu.edu/pub_files/pub3/baker_simon_2002_3/baker_simon_2002_3.pdf) and [link](https://www.ri.cmu.edu/pub_files/pub3/baker_simon_2003_3/baker_simon_2003_3.pdf).


### Q1.1:  Lucas-Kanade Forward Additive Alignment with Translation Only (10 PT write-up, 20 PT implementation)
Write the function with the following function signature:

```
            p = LucasKanade(It, It1, rect, thresh, maxIt)
```
that computes the optimal local motion $p$ represented by translation (motion in x and y directions) from frame $I_t$ to frame $I_{t+1}$that minimizes

$$
\begin{gathered}
\mathcal{L}=\sum_{\mathbf{x}}[\mathbf{T}(\mathbf{x})-\mathbf{I}(\mathbf{W}(\mathbf{x} ; \mathbf{p}))]^{2}. 
\end{gathered}
$$

"It" is the image frame $I_t$, "It1" is the image frame $I_{t+1}$, and "rect" is a $4×1$ vector that represents a rectangle (bounding box) on the image frame $I_t$. The four elements of the rectangle are $[x1, y1, x2, y2]$, where $(x1, y1)$ is the top-left corner and $(x2, y2)$ is the bottom-right corner of the bounding box. "thresh" and "maxIt" control when the algorithm should stop, depending on if dp is too small or the maximum number of iterations has been reached, respectively.

Hints:
- To deal with the fractional movement of the template in the bounding box, you will need to interpolate the image using scipy.interpolate.RectBivariateSpline. The same function can also be used to compute the gradient of an image at a point location.
- You will need to iterate the estimation in Equation 10 until the change in warp parameters $(dx, dy)$ is below a threshold or the number of iterations is too large.
- You can use np.linalg.lstsq to solve the least square problem in Equation 10.

<span style='color:red'>**Output:**</span> In your write-up: Please include the results of the algorithm on all five videos we have provided along with your code.

In [2]:
def rect_indices(rect):
    """
    TODO
    """
    # x = axis1, y = axis0
    x1, y1, x2, y2 = rect
    # Round to the nearest integer (should essentially be an integer to begin with)
    a0_range = int(y2 - y1 + 0.5)
    a1_range = int(x2 - x1 + 0.5)
    # TODO
    a0_values = np.arange(y1, y2).reshape(-1, 1)
    a0_vector = np.hstack((a0_values, np.ones(a0_values.shape) * x1))
    a1_values = np.arange(x1, x2).reshape(-1, 1)
    a1_vector = np.hstack((np.ones(a1_values.shape) * y1, a1_values))
    return a0_vector, a1_vector


def translate_warp(p, rect, image, dx=0, dy=0):
    a0_vector, a1_vector = rect_indices(rect)
    # This is the translational warp, specifically
    a0_warped = (a0_vector + p)[:, 0]
    a1_warped = (a1_vector + p)[:, 1]
    # Return specific warped values
    spline = RectBivariateSpline(
        x=np.array(range(image.shape[0])),
        y=np.array(range(image.shape[1])),
        z=image,
    )
    query = spline(
        x=a0_warped,
        y=a1_warped,
        dx=dx,
        dy=dy,
    )
    return query

In [3]:
def LucasKanade(It, It1, rect, thresh=.025, maxIt=100):
    
    '''
    Q1.1: Lucas-Kanade Forward Additive Alignment with Translation Only
    
      Inputs: 
        It: template image
        It1: Current image
        rect: Current position of the object
        (top left, bottom right coordinates, x1, y1, x2, y2)
        thresh: Stop condition when dp is too small
        maxIt: Maximum number of iterations to run
        
      Outputs:
        p: movement vector dx, dy
    '''
    import cv2
    
    # Set thresholds (you probably want to play around with the values)
    p = np.zeros(2) # dx, dy
    threshold = thresh
    maxIters = maxIt
    i = 0

    # # Use to visualize the images and the starting bounding boxes
    # plotim = It.copy()
    # cv2.rectangle(img=plotim, pt1=(x1, y1), pt2=(x2, y2), color=255, thickness=3)
    # a0vec, a1vec = rect_indices(rect)
    # plotim[a0vec.astype(int)[:, 0], a0vec.astype(int)[:, 1]] = 0
    # plotim[a1vec.astype(int)[:, 0], a1vec.astype(int)[:, 1]] = 0
    # plt.imshow(plotim)
    # plt.show()
    # plt.imshow(It1)
    # plt.show()

    # ----- TODO -----
    # YOUR CODE HERE

    # Get the template patch once
    template = translate_warp(p=np.array([0, 0]), rect=rect, image=It).flatten()
    # # Compute the gradient over the whole current image
    # It1_x = cv2.Scharr(src=It1, ddepth=-1, dx=1, dy=0)
    # It1_y = cv2.Scharr(src=It1, ddepth=-1, dx=0, dy=1)

    # # Use to visualize the patches
    # plt.imshow(It[y1:y2, x1:x2]); plt.show()
    # plt.imshow(It1[y1:y2, x1:x2]); plt.show()
    # plt.imshow(translate_warp(np.array([10, -20]), rect, It1)); plt.show()
    # import ipdb; ipdb.set_trace()

    for _ in range(maxIters):
        # if _ % 20 == 0: print(_)

        # Get warps with the new p
        warped_patch = translate_warp(p, rect, It1)
        # Unsolved mystery - is the interpolated cv2 filter on Ix Iy bad? It don't match
        # warped_Ix = translate_warp(p, rect, It1_x)
        # warped_Iy = translate_warp(p, rect, It1_y)
        warped_Ix = translate_warp(p, rect, It1, dx=1)
        warped_Iy = translate_warp(p, rect, It1, dy=1)
        # import ipdb; ipdb.set_trace()
        
        # In this specific case, dW/dp = I and we can get grad(Im)dW/dp
        # just with grad(Im)
        grad_I_dWdp = np.vstack((warped_Ix.flatten(), warped_Iy.flatten()))
        
        # Get the T(x) - I(W(x, p)) term (template is pre-flattened)
        error = template - warped_patch.flatten()
        
        # Combine terms and sum
        descent_vector = np.sum(grad_I_dWdp * error, axis=1)
        # Did some whiteboard math to convince me this was equivalent
        H = grad_I_dWdp @ grad_I_dWdp.T

        delta_p = np.linalg.inv(H) @ descent_vector
        p += delta_p
        if np.allclose(delta_p, 0, atol=threshold):
            print(f"Breaking! p: {p}, delta_p: {delta_p}")
            break
        # print(f"p: {p}, delta_p: {delta_p}")

    return p


# # Exploration for what RectBivariateSpline is doing
# x = np.array([ 1, 1.5, 2, 4,   6, 6.5,   7, 7.5, 8, 8.5, 9, 9.5])
# y = np.array([0.5, 1, 1.5,   2, 2.5,   3, 5,   7, 8, 9])
# z = np.outer(np.array(range(12)), np.array(range(10, 20)))
# spline = RectBivariateSpline(range(12), range(10), z)
# output = spline(x, y)
# print(x)
# print(y)
# print(z)
# print(output)

# # Visualize images
# for data_name in ["car1", "car2", "landing", "race", "ballet"]:
#     print(f"data_name: {data_name}")
#     data = np.load('./data/%s.npy' % data_name)
#     if data_name == 'car1':      initial = np.array([170, 130, 290, 250])
#     elif data_name == 'car2':    initial = np.array([59, 116, 145, 151])
#     elif data_name == 'landing': initial = np.array([440, 80, 560, 140])
#     elif data_name == 'race':    initial = np.array([170, 270, 300, 370])
#     elif data_name == 'ballet':  initial = np.array([700, 210, 775, 300])
#     else: assert False, 'the data name must be one of (car1, car2, landing, race, ballet)'
#     numFrames = data.shape[2]
#     It = data[:,:,0]
#     It1 = data[:,:,1]
#     p = LucasKanade(It, It1, initial).astype(int)

#     import cv2
#     x1, y1, x2, y2 = initial
#     plotim = It1.copy()
#     cv2.rectangle(img=plotim,
#                   pt1=(x1+p[0], y1+p[1]),
#                   pt2=(x2+p[0], y2+p[1]),
#                   color=255,
#                   thickness=3)
#     plt.imshow(plotim)
#     plt.show()

In [4]:
# Test your algorithm and visualize results!

# Load data
data_name = 'landing' # could choose from (car1, car2, landing, race, ballet)
# for data_name in ["car1", "car2", "landing", "race", "ballet"]:
for data_name in ["race", "ballet"]:
    data = np.load('./data/%s.npy' % data_name)

    # obtain the initial rect with format (x1, y1, x2, y2)
    if data_name == 'car1':
        initial = np.array([170, 130, 290, 250])
    elif data_name == 'car2':
        initial = np.array([59, 116, 145, 151])
    elif data_name == 'landing':
        initial = np.array([440, 80, 560, 140])
    elif data_name == 'race':
        initial = np.array([170, 270, 300, 370])
    elif data_name == 'ballet':
        initial = np.array([700, 210, 775, 300])
    else:
        assert False, 'the data name must be one of (car1, car2, landing, race, ballet)'

    numFrames = data.shape[2]
    w = initial[2] - initial[0]
    h = initial[3] - initial[1]

    # loop over frames
    rects = []
    rects.append(initial)

    for i in range(numFrames-1):

        It = data[:,:,i]
        It1 = data[:,:,i+1]
        rect = rects[i]

        # run algorithm and collect rects
        dx, dy = LucasKanade(It, It1, rect)
        newRect = np.array([rect[0] + dx, rect[1] + dy, rect[0] + dx + w, rect[1] + dy + h])
        rects.append(newRect)

        # Visualize
        fig = plt.figure(1, figsize=(12, 12))
        ax = fig.add_subplot(111)
        ax.add_patch(patches.Rectangle((rect[0], rect[1]), rect[2]-rect[0]+1, rect[3]-rect[1]+1, linewidth=2, edgecolor='red', fill=False))
        plt.imshow(It1, cmap='gray')
        plt.savefig(f"./vis/{data_name}_{i}.png")
        plt.close(fig)

Breaking! p: [0.37920347 5.52193019], delta_p: [-0.00095979  0.01880636]
Breaking! p: [2.73135394 4.16481531], delta_p: [-0.00242135  0.01717992]
Breaking! p: [ 2.53401557 -0.5298332 ], delta_p: [0.01794148 0.00440703]
Breaking! p: [ 1.50664002 -0.09609217], delta_p: [0.01013044 0.00045361]
Breaking! p: [ 3.21114027 -1.73202489], delta_p: [ 0.00117668 -0.01739311]
Breaking! p: [-0.82097951  0.16799589], delta_p: [-0.01099948  0.01081539]
Breaking! p: [-1.30730147 -1.1553124 ], delta_p: [-0.00401281 -0.01796872]
Breaking! p: [ 1.22819409 -3.79279076], delta_p: [-0.00078148 -0.02486553]
Breaking! p: [-2.31297729 -4.92662942], delta_p: [ 0.00054272 -0.01533204]
Breaking! p: [2.78758924 0.31958302], delta_p: [0.00897312 0.0097616 ]
Breaking! p: [-0.26242634  3.69097031], delta_p: [-0.00058033  0.01347424]
Breaking! p: [-5.07313979  2.80831889], delta_p: [-0.00067556  0.02196012]
Breaking! p: [3.65782262 5.71316164], delta_p: [-4.64203467e-05  1.79943782e-02]
Breaking! p: [-2.11264394  0.09

In [ ]:
# For some transparency: we evaluate on multiple frames in a given video starting from the first frame.
# We then compare against the reference implementation and calculate the sum of all differences.
# You should not need to tune anything for the autograding. We pass in the same hyperparameters for you.


### Q1.2:  Lucas-Kanade Forward Additive Alignment with Affine Transformation (10 PT write-up, 20 PT implementation)
Assuming that the warp is translation-only is quite limiting. Now we will assume that the warp takes on the form of an arbitrary affine transformation. Write the function with the following function signature:

```
            M = LucasKanadeAffine(It, It1, rect):
```
that computes the optimal local motion represented by a $2x3$ affine transformation matrix $M$ from frame $I_t$ to frame $I_{t+1}$that minimizes

$$
\begin{gathered}
\mathcal{L}=\sum_{\mathbf{x}}[\mathbf{T}(\mathbf{x})-\mathbf{I}(\mathbf{W}(\mathbf{x} ; \mathbf{p}))]^{2}. 
\end{gathered}
$$

The inputs are structured identically to the previous problem.

<span style='color:red'>**Output:**</span> In your write-up: Please include the results of the algorithm on all five videos we have provided along with your code.

In [ ]:
def LucasKanadeAffine(It, It1, rect, thresh=.025, maxIt=100):
    '''
    Q1.2: Lucas-Kanade Forward Additive Alignment with Affine MAtrix
    
      Inputs: 
        It: template image
        It1: Current image
        rect: Current position of the object
        (top left, bottom right coordinates, x1, y1, x2, y2)
        thresh: Stop condition when dp is too small
        maxIt: Maximum number of iterations to run
        
      Outputs:
        M: Affine mtarix (2x3)
    '''

    # Set thresholds (you probably want to play around with the values)
    M = np.zeros((2,3))
    threshold = thresh
    maxIters = maxIt
    i = 0
    x1, y1, x2, y2 = rect
    
    # ----- TODO -----
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return M


In [ ]:
# Test your algorithm and visualize results!

# Load data
data_name = 'landing' # could choose from (car1, car2, landing, race, ballet)
data = np.load('./data/%s.npy' % data_name)

# obtain the initial rect with format (x1, y1, x2, y2)
if data_name == 'car1':
    initial = np.array([170, 130, 290, 250])   
elif data_name == 'car2':
    initial = np.array([59, 116, 145, 151])    
elif data_name == 'landing':
    initial = np.array([440, 80, 560, 140])     
elif data_name == 'race':
    initial = np.array([170, 270, 300, 370])
elif data_name == 'ballet':
    initial = np.array([700, 210, 775, 300])     
else:
    assert False, 'the data name must be one of (car1, car2, landing, race, ballet)'

numFrames = data.shape[2]
w = initial[2] - initial[0]
h = initial[3] - initial[1]

# loop over frames
rects = []
rects.append(initial)

for i in range(numFrames-1):

    It = data[:,:,i]
    It1 = data[:,:,i+1]
    rect = rects[i]

    # run algorithm and collect rects
    M = LucasKanadeAffine(It, It1, rect)
    corners = np.array([[rect[0], rect[1], 1], 
                        [rect[2], rect[3], 1]]).transpose()
    newRect = np.matmul(M, corners).transpose().reshape((4, ))
    rects.append(newRect)

    # Visualize
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    ax.add_patch(patches.Rectangle((rect[0], rect[1]), rect[2]-rect[0]+1, rect[3]-rect[1]+1, linewidth=2, edgecolor='red', fill=False))
    plt.imshow(It1, cmap='gray')
    plt.show()
    ax.clear()


In [ ]:
# For some transparency: we evaluate on multiple frames in a given video starting from the first frame.
# We then compare against the reference implementation and calculate the sum of all differences.
# You should not need to tune anything for the autograding. We pass in the same hyperparameters for you.
